# Table of Contents

1. [Problem Statement](#Problem-Statement)
2. [Background Information](#Background-Information)
3. [Executive Summary](#Executive-Summary)
4. [Imports](#Imports)
5. [Soccer Match Preliminary EDA](#Soccer-Match-Preliminary-EDA)
6. [Building Helper Functions](#Building-Helper-Functions)
7. [Processing All Event Data](#Processing-All-Event-Data)
    1. [Concatinating All Event Data](#Concatinating-All-Event-Data)
    2. [Extracting Nested Dictionary Information](#Extracting-Nested-Dictionary-Information)
    3. [Exporting Team Data](#Exporting-Team-Data)

## Problem Statement

As team weaknesses are discovered, what players can fill the roles that are needed? I will be using unsupervised clustering algorithms to recommend players, with similar playing styles, to supplement a team's overall weaknesses. Player movement heatmap images and team movement heathamps images will be our main source of data.  

Due to limited time constraints, this project focus on generating images of player movement through the course of a season. The dataset created will be used for the next phase of this project. 


## Background Information

Currently, signing a new soccer player is based a lot on name recognition, goal scoring percentile, and sometimes just hype. Building a team around world class players is what everyone wants but sometimes, even world class players have to adjust to a new position if their natural position is already filled by another superstar. The likes of Messi, Suarez, and Griezmann are all superstars but have difficulty building chemistry in the attacking third of the pitch. This is partly because Griezman needs to adapt to a wider position than he was used to naturally playing at his former club. At his former club, Griezman would be the focal point of attack and now at Barcelona, he's not. Instead he is playing on the wing or as a striker when Luis Suarez is injured. 

This isn't just limited to Griezman but other soccer players as well. Hirving Lozano is currently Napoli's most expensive signing at $\$46.5$ million dollars. Pundits in Italy have been harsh critics about this transfer since he hasn't been able to make an impact like he did at his former club. We can also look at Neymar's record \$263 million transfer fee to PSG. He hasn't been able to make an impact on his team's performance overall. Neymar's case might be a bit different because there are problems unrelated to his football abilities that have hindered his performance but the point is the same. 

How can we, as a club manager or owner know that a player will be a great addition to their team? Just because players have great talent doesn't necessarily make them the perfect fit. 

In Barcelona's case, if someone else who was naturally playing as a Left Winger would come in to replace Neymar, there wouldn't be a need to put so much pressure on Griezman to quickly adapt to a new role this isn't his natural role. He can still do the job well but he isn't as efficient as he was at his former club. It is frustrating to see a player like Griezman valued at $135 million not produce the results that are expected of him, from a club manager's perspective. Equally important, the player feels frustrated in not being about to produce similar results and will take longer to adaptat. The longer the adaptation period, the more impatient the fans, pundits, club owners, and even the player themself will get. The player's need for time to adapt to a new role, and the manager's need to see positive results as soon as possible are a difficult balance. 

## Executive Summary

Statsbomb has a public repo that is periodically updated with public event data for particular soccer matches that they release. The data I was working with initially consisted of 2.27 million rows once it was aggregated together. This was a lot of data to handle given the time constraints and data cleaning that needed to happen before even analyzing data. Upon reevaluation, I decided to focus on Barcelona's team since approximately 875k rows were pertaining to Barcelona games.

Data cleaning consisted of unpacking information stored as python dictionary equivalents within the json files. Some data contained nested dictionaries and data wrangling became much of the time consuming portion of the project. Functions were created in order to automate the process. However, the overall cleaning process was limited to Barcelona data due to the limited processing power of my computer and time constraints. Additionally, we narrowed down to the basic test cases for each Starting-XI players in a given Barcelona match. This serves as a proof of concept and the process becomes smoother when scaling up to larger quantities of matches per player and overall data. 

In the end, GIFs were created in order for us to see player activity density every 5 minutes per game. When scaling to include the activity density for a player across a whole season, the "natural" position of that player will be revealed. This is because one game is not enough of a sample size to determine a player's natural position. A player can be playing a different position in the next game due to another player's injury or a tactical experiment. Over the course of the season, we are able to determine what postions were played effectivly and consistently based on heatmap image intensity. 


## Imports

In [1]:
import pandas as pd 
import numpy as np

## Soccer Match Preliminary EDA

We need to look at one particular file in order to familiarize ourselves with how the data for each event is stored.

In [2]:
event_data = pd.read_json("../open-data/data/events/19714.json")

In [3]:
event_data.info(verbose = True) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 35 columns):
id                 3491 non-null object
index              3491 non-null int64
period             3491 non-null int64
timestamp          3491 non-null datetime64[ns]
minute             3491 non-null int64
second             3491 non-null int64
type               3491 non-null object
possession         3491 non-null int64
possession_team    3491 non-null object
play_pattern       3491 non-null object
team               3491 non-null object
duration           2632 non-null float64
tactics            12 non-null object
related_events     3215 non-null object
player             3471 non-null object
position           3471 non-null object
location           3461 non-null object
pass               947 non-null object
carry              771 non-null object
under_pressure     620 non-null float64
ball_receipt       177 non-null object
counterpress       127 non-null float64
dribble       

It looks like we need to figure out what null values imply for each column. By the look of the column names with limited values, my initial assumption is that certain types of events such as fouls or substitutions don't happen that often throughout the game. Regardless, lets look at our data now

In [4]:
event_data

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,...,goalkeeper,interception,foul_committed,duel,50_50,off_camera,foul_won,substitution,block,clearance
0,caa84999-ca96-408d-93b5-c8f74d832237,1,1,2019-12-02 00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 971, 'name': 'Chelsea FCW'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9fffa92c-6397-4dd4-9ff4-5c560bd84e53,2,1,2019-12-02 00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 971, 'name': 'Chelsea FCW'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,906917ac-7840-41cf-9e76-17df27d3edb3,3,1,2019-12-02 00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 971, 'name': 'Chelsea FCW'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,430a8774-f8e7-4691-a0fd-4eeb0491d5d0,4,1,2019-12-02 00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 971, 'name': 'Chelsea FCW'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,883510b1-40a0-49be-a590-ab2f1dcc6bdf,5,1,2019-12-02 00:00:00.903,0,0,"{'id': 30, 'name': 'Pass'}",2,"{'id': 746, 'name': 'Manchester City WFC'}","{'id': 9, 'name': 'From Kick Off'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,71a089de-2fcc-46a7-96e4-d470da0037d3,3487,2,2019-12-02 00:49:25.587,94,25,"{'id': 16, 'name': 'Shot'}",226,"{'id': 971, 'name': 'Chelsea FCW'}","{'id': 3, 'name': 'From Free Kick'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3487,7e972c3f-4a6a-41fa-bf04-afa2867538ba,3488,2,2019-12-02 00:49:26.026,94,26,"{'id': 6, 'name': 'Block'}",226,"{'id': 971, 'name': 'Chelsea FCW'}","{'id': 3, 'name': 'From Free Kick'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3488,ffc366c5-35c0-45f5-9b66-f0d94fc45885,3489,2,2019-12-02 00:49:26.573,94,26,"{'id': 23, 'name': 'Goal Keeper'}",226,"{'id': 971, 'name': 'Chelsea FCW'}","{'id': 3, 'name': 'From Free Kick'}",...,"{'end_location': [13.0, 40.0], 'position': {'i...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3489,e76aee8d-5bbc-43a7-ae07-04867575d7c8,3490,2,2019-12-02 00:49:29.284,94,29,"{'id': 34, 'name': 'Half End'}",226,"{'id': 971, 'name': 'Chelsea FCW'}","{'id': 3, 'name': 'From Free Kick'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


It looks like some columns have values that contain dictionaries and that might get messy. I will need to identify all the columns that have a dictionary as a value. 

## Building Helper Functions

The `get_dict_names` function will take in a column and a dataframe and return a dictionary of keys that were found in values that were dictionaries within the column

In [5]:
def get_dict_names(col,df):
    #pass_keys is where I will store all the keys I find in each dictionary in a particular column
    #Note that this list will be redundant since most keys will be the same but there are some exceptions to this rule
    pass_keys = []
    
    #for each row in the column, we will check if the row contains a dictionary since there are columns with both NaN
    #values and dictionaries in the same column! If there is a dictionary found then we will add the keys of that
    #dictionary to our list of keys 
    for row in df[col]:
        if type(row) == dict:
            pass_keys.extend(list(row.keys()))
    
    #We will return the dictionary of keys since some dictionaries may contain keys that 
    #may be in one dictionary but not in another 
    return {key:[] for key in set(pass_keys)} 

The `unwrap_col` function will take in a col and a dataframe and return dataframe with content of the columns that were unwrapped and a list of colums (either one or empty). The goal is to be able to either append the list to our original dataframe if everything looks good or if there are more dictionaries foun in this dataframe, we can repeat the process to this dataframe instead of our original dataframe

In [6]:
def unwrap_col(col, df):
    #list of columns I will need to drop in the original dataframe 
    drop_col = []
    
    #set of keys that we know are in the column of dictionaries
    keys_dict = get_dict_names(col,df)
    
    #for each row in the column provided, if the type of that row is a dictionary, add that column to drop_col list
    for row in df[col]: 
        if type(row) == dict:
            drop_col.append(col)
            
            #Once we know a row is a dictionary, we will loop through each key in our keys_dict. If the key is found
            #in the row, we will append the value of that key in the column to our keys_dict. If not, we will add an 
            #empty string as a place holder since we want the length of our final list to be the same as our df
            for key in keys_dict: 
                if key in row:
                    keys_dict[key].append(row[key])
                else: 
                    keys_dict[key].append("")
                    
        #if it turns out our row is null or is not a dictionary, we will add an empty string as a place holder
        #to each key in our keys_dict
        else:
            for key in keys_dict:
                keys_dict[key].append("")
    
    #We will return a dataframe with the columns that were unwrapped. 
    #Since the drop cols will have redundant info, we will take the set, and will only have one column 
    #and turn that set into a list. If there are no dictionaries, we should have an empty list
    return pd.DataFrame(keys_dict).rename(columns = {key:f'{col}_{key}' for key in keys_dict}) , list(set(drop_col))
    

## Processing All Event Data

We are now ready to create a Data Cleaning Pipeline for all of our files!

### Concatinating All Event Data

In [7]:
import os

The point of this is to eventually have all information from a specific team grouped together, across multiple events. The first step is to concatinate all the data we have into a master dataframe

In [8]:
files = os.listdir('../open-data/data/events/')

full_df = pd.concat([pd.read_json(f'../open-data/data/events/{file}') for file in files], sort = False, ignore_index = True)

In [9]:
full_df.shape

(2274242, 42)

The first file that we explored might have values that were not tracked during the time that it was written. New data have been tracked and added by Statsbomb since their initial release of public data. 

In [10]:
set(full_df.columns)- set(event_data.columns)

{'bad_behaviour',
 'half_end',
 'half_start',
 'injury_stoppage',
 'miscontrol',
 'out',
 'player_off'}

A quick peep at our dataframe:

In [11]:
full_df.head()

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,...,substitution,injury_stoppage,block,bad_behaviour,out,miscontrol,50_50,half_start,half_end,player_off
0,92f7b2d2-d048-4bbc-b8ff-67dcfc328fb2,1,1,2019-12-02 00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 974, 'name': 'Reading WFC'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e61277b1-87a1-40e5-becc-dd727c821920,2,1,2019-12-02 00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 974, 'name': 'Reading WFC'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bfcb5195-fef1-4078-b76a-2240dcdb78d7,3,1,2019-12-02 00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 974, 'name': 'Reading WFC'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,503d06c8-5441-43b3-9aa5-7ea68ecabad2,4,1,2019-12-02 00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 974, 'name': 'Reading WFC'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4b92ae51-2702-4d07-a9e3-848fa1170cb3,5,1,2019-12-02 00:00:00.080,0,0,"{'id': 30, 'name': 'Pass'}",2,"{'id': 974, 'name': 'Reading WFC'}","{'id': 9, 'name': 'From Kick Off'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Extracting Nested Dictionary Information

As we noticed earlier, we have some values that are dictionaries and some of those dictionaries are nested as well. We need to be able to readily access that information and get rid of those dictionaries. Possible reasons for having data nested like that could be easy of storage and organization. 

We will use the `unpack_df` function below to unpack these dictionaries

In [12]:
def unpack_df(df):
    #Initial number of Columns in df
    cols_left = df.shape[1]
    
    #Columns we will drop after dictionary 
    cols_drop = []
    
    for col in df:
        print(f'{cols_left} Columns Left to Unpack.', end = '\r')
        
        #colum to drop if there is a dictionary value present
        new_df, drop_col = unwrap_col(col, df)
        #repeating the process for a nested dictionary, if there is one
        temp_dict = {}
        for sub_col in new_df:
            temp_dict[sub_col], drop_sub_col = unwrap_col(sub_col, new_df)
            cols_drop.extend(drop_sub_col)
        for sub_col in temp_dict:
            new_df = new_df.join(temp_dict[sub_col])
        df = df.join(new_df)
        cols_drop.extend(drop_col)
        cols_left -= 1
        
    print('Unpacking Complete')
    
    df.drop(columns = cols_drop, inplace = True)
    
        
    return df

In [13]:
full_df = unpack_df(full_df)

42 Columns Left to Unpack.
41 Columns Left to Unpack.
40 Columns Left to Unpack.
39 Columns Left to Unpack.
38 Columns Left to Unpack.
37 Columns Left to Unpack.
36 Columns Left to Unpack.
35 Columns Left to Unpack.
34 Columns Left to Unpack.
33 Columns Left to Unpack.
32 Columns Left to Unpack.
31 Columns Left to Unpack.
30 Columns Left to Unpack.
29 Columns Left to Unpack.
28 Columns Left to Unpack.
27 Columns Left to Unpack.
26 Columns Left to Unpack.
25 Columns Left to Unpack.
24 Columns Left to Unpack.
23 Columns Left to Unpack.
22 Columns Left to Unpack.
21 Columns Left to Unpack.
20 Columns Left to Unpack.
19 Columns Left to Unpack.
18 Columns Left to Unpack.
17 Columns Left to Unpack.
16 Columns Left to Unpack.
15 Columns Left to Unpack.
14 Columns Left to Unpack.
13 Columns Left to Unpack.
12 Columns Left to Unpack.
11 Columns Left to Unpack.
10 Columns Left to Unpack.
9 Columns Left to Unpack.
8 Columns Left to Unpack.
7 Columns Left to Unpack.
6 Columns Left to Unpack.
5 Col

A quick peep at our concatinated dataframe. Note that there are lists but that's okay because we will be using that information later on

In [14]:
full_df.head()

,id,index,period,timestamp,minute,second,possession,duration,related_events,location,...,block_offensive,block_deflection,bad_behaviour_card_name,bad_behaviour_card_id,miscontrol_aerial_won,50_50_outcome_name,50_50_outcome_id,half_start_late_video_start,half_end_early_video_end,player_off_permanent
0,92f7b2d2-d048-4bbc-b8ff-67dcfc328fb2,1,1,2019-12-02 00:00:00.000,0,0,1,0.0000,NaN,NaN,...,,,,,,,,,,
1,e61277b1-87a1-40e5-becc-dd727c821920,2,1,2019-12-02 00:00:00.000,0,0,1,0.0000,NaN,NaN,...,,,,,,,,,,
2,bfcb5195-fef1-4078-b76a-2240dcdb78d7,3,1,2019-12-02 00:00:00.000,0,0,1,0.0000,[503d06c8-5441-43b3-9aa5-7ea68ecabad2],NaN,...,,,,,,,,,,
3,503d06c8-5441-43b3-9aa5-7ea68ecabad2,4,1,2019-12-02 00:00:00.000,0,0,1,0.0000,[bfcb5195-fef1-4078-b76a-2240dcdb78d7],NaN,...,,,,,,,,,,
4,4b92ae51-2702-4d07-a9e3-848fa1170cb3,5,1,2019-12-02 00:00:00.080,0,0,2,1.7322,[e39d5015-8ff8-429e-949f-a7d27bfd8de5],"[60.0, 40.0]",...,,,,,,,,,,


In [22]:
full_df['possession_team_name'].value_counts()

Barcelona              874826
Manchester City WFC     55791
Arsenal WFC             55450
Chelsea FCW             49619
Birmingham City WFC     46429
                        ...  
Chelsea LFC              1545
Cádiz                    1422
Real Murcia              1418
Deportivo Alavés         1222
Gimnàstic Tarragona      1037
Name: possession_team_name, Length: 117, dtype: int64

We will be analyzing Barcelona in the next notebook since we have the most information on Barcelona.

### Exporting Team Data

In [16]:
teams = set(full_df['possession_team_name'])

In [17]:
teams_left = len(teams)
print(f'We will need to make {teams_left} data CSV files')
for team in teams:
    team_df = full_df[full_df['possession_team_name'] == team]
    team_df.to_csv(f'../open-data/data/teams/{team}_data.csv', index = False)
    print(f'{team} data file has been created.')
    teams_left -= 1 
    print(f'There are {teams_left} CSV files left to create.')
    print('=================================\n')

We will need to make 117 data CSV files
Jamaica Women's data file has been created.
There are 116 CSV files to create.

Portland Thorns data file has been created.
There are 115 CSV files to create.

Numancia data file has been created.
There are 114 CSV files to create.

Argentina data file has been created.
There are 113 CSV files to create.

Chelsea LFC data file has been created.
There are 112 CSV files to create.

Reading WFC data file has been created.
There are 111 CSV files to create.

Albacete data file has been created.
There are 110 CSV files to create.

Eibar data file has been created.
There are 109 CSV files to create.

Switzerland data file has been created.
There are 108 CSV files to create.

Costa Rica data file has been created.
There are 107 CSV files to create.

Villarreal data file has been created.
There are 106 CSV files to create.

Iran data file has been created.
There are 105 CSV files to create.

Italy Women's data file has been created.
There are 104 CSV fil

Real Murcia data file has been created.
There are 41 CSV files to create.

Korea Republic Women's data file has been created.
There are 40 CSV files to create.

China PR Women's data file has been created.
There are 39 CSV files to create.

Netherlands Women's data file has been created.
There are 38 CSV files to create.

Brazil Women's data file has been created.
There are 37 CSV files to create.

Bristol City WFC data file has been created.
There are 36 CSV files to create.

Poland data file has been created.
There are 35 CSV files to create.

Senegal data file has been created.
There are 34 CSV files to create.

Getafe data file has been created.
There are 33 CSV files to create.

Orlando Pride data file has been created.
There are 32 CSV files to create.

Real Madrid data file has been created.
There are 31 CSV files to create.

Liverpool WFC data file has been created.
There are 30 CSV files to create.

Córdoba data file has been created.
There are 29 CSV files to create.

Osasuna

In [21]:
pd.DataFrame(data = teams, columns = ['team_name']).to_csv('./teams_included.csv', index= False)

All data is separated and ready to be imported in the next notebook. However, we will be focusing on Barcelona as our test case. 